In [1]:

import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from graphviz import *
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
# import importlib
# importlib.reload(hoge)
from my_library.horse import *

# path

In [2]:
path_ubu = '/home/hipro/デスクトップ/Horse/Data/20_21'
path_win2 = '/Users/rince/Desktop/Horse/Data/saishin2/'
path_win = '/Users/rince/Desktop/Horse/Data/saishin/'
path_win = '/Users/Owner/Desktop/program/Horse/Data/saishin/'
path_win2 = '/Users/Owner/Desktop/program/Horse/Data/saishin2/'
path_ft = '/Users/Owner/Desktop/Horse/ft_data/peds_ft.txt'

# load data

In [4]:
results = load_csv(path_win+'results.csv')
horse_results = load_csv(path_win+'horse_results.csv')
peds = load_csv(path_win+'peds.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_win+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

In [4]:
results.tail()

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,course_len,weather,race_type,ground_state,date,horse_id,jockey_id
202202010612,9,8,13,スカイトレイル,牝3,50.0,角田大和,1:10.1,クビ,6.5,3.0,462(+4),[西] 西村真幸,1200,曇,芝,良,2022年6月26日,2019103814,1186
202202010612,10,8,12,クローリスノキセキ,牝4,52.0,小林凌大,1:10.2,1/2,14.4,4.0,446(-4),[東] 西田雄一,1200,曇,芝,良,2022年6月26日,2018103618,1177
202202010612,11,6,9,ルワンジュ,牡3,50.0,古川奈穂,1:10.7,3,150.2,13.0,484(+2),[東] 上原博之,1200,曇,芝,良,2022年6月26日,2019105798,1190
202202010612,12,2,2,ナムラデイリリー,牝3,51.0,泉谷楓真,1:11.0,2,53.2,10.0,474(+4),[西] 大橋勇樹,1200,曇,芝,良,2022年6月26日,2019102535,1182
202202010612,13,1,1,ケヴィンズクロス,牡4,57.0,黛弘人,1:11.4,2.1/2,18.8,5.0,508(+2),[東] 菊沢隆徳,1200,曇,芝,良,2022年6月26日,2018103001,1109


# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [5]:
ll = LearnLGBM(peds,results,horse_results)
ll.learn_lgb()

  0%|          | 0/18009 [00:00<?, ?it/s]

In [ ]:
train_query = ll.x_train.groupby(ll.x_train.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
ll.learn_lgb2(train,lgbm_params=lgbm_params)

In [ ]:
# ll = LearnLGBM(peds,results,horse_results)
# ll.get_train_data()


val_x = ll.x_test.iloc[:-10000]
val_y = ll.y_test.iloc[:-10000]
test_query = val_x.groupby(val_x.index).size()
train_query = ll.x_train.groupby(ll.x_train.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
valid = lgb.Dataset(val_x, val_y, reference=train, group=test_query)

# 正解ラベル　変えた

In [76]:
def split_data(df, test_size=0.2, rank_learning=True):
    """
    データを学習データと, 訓練データに分ける関数
    """
    df_ = df.copy()
    df_['rank'] = df_['rank'].map(lambda x: int((1/x)*10) if x<4 else(0))
        
    sorted_id_list = df_.sort_values("date").index.unique()
    train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
    train = df_.loc[train_id_list]#.drop(['date'], axis=1)
    test = df_.loc[test_id_list]#.drop(['date'], axis=1)
    return train, test

    
train, test = split_data(pt.r.data_c.fillna(0),test_size=0.2,rank_learning=False)
x_train = train.drop(['rank', 'date','単勝'], axis=1)
y_train = train['rank']
x_test = test.drop(['rank', 'date','単勝'], axis=1)
y_test = test['rank']
train_query = x_train.groupby(x_train.index).size()
train = lgb.Dataset(x_train, y_train, group=train_query)
# val_y = y_test.iloc[:-10000]
test_query = x_test.groupby(x_test.index).size()
valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

In [84]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                # 'lambdarank_truncation_level': 37,
                #  'learning_rate': 0.01271180569095827,
                'lambdarank_truncation_level': 10,
                'learning_rate': 0.02273417953255777,
                'n_estimators': 97,
                'num_leaves': 42,
                'force_col_wise':True
            }



In [ ]:
import warnings

warnings.filterwarnings("ignore")
ll.learn_lgb2(train,lgbm_params=lgbm_params)

In [67]:
def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用
    # train = lgb.Dataset(x_train, y_train, group=train_query)
    # valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

    
    lgb_clf = lgb.train(
        params,
        train,
        num_boost_round=1000,
        valid_sets=valid,
        valid_names=['valid'],
        early_stopping_rounds=20,
        verbose_eval=5,
        evals_result=lgb_results,
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
#         best paramsの返り値は, choiceだとindexか?
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',range(1,50)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        # n_estimators == num_iterations
        # 'num_iterations': hp.choice('num_iterations',range(50,120)),
        # 'num_leaves': hp.choice('num_leaves',range(20,50)),
        'boosting_type': 'gbdt',
        'random_state': 777
    }

    max_evals = 25      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)

In [ ]:
trials = Trials()
optimize(trials)

# Simulate


In [ ]:

for odds in [1.1,2.0,3.0,4.0,5.0,6.0,7.0]:
    print()
    sl = RankSimulater(ll.model)
    print("odds",odds)
    sl.calc_tansho(ll.r.data_c.iloc[-10000:],return_tables,odds=odds)



In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrenpuku(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrentan(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

# race_id_list

In [3]:

202205021011
race_id_list = ['2022050305{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022050306{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022050307{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022050308{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022090301{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022090302{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022090303{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022090304{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022020103{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022020104{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022020105{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022020106{}'.format(str(i).zfill(2)) for i in range(1,13)]



# Predictor

In [86]:
pt = Predictor(peds,results,horse_results,race_id_list)
# ********* test_size = 0  : ずっと0にしてなかった...
pt.learn_lgb(lgbm_params=lgbm_params,test_size=0)


pe finish
pe regularizrd



hr finish


/Users/rince/Library/Python/3.7/lib/python/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Total Bins 48797
[LightGBM] [Info] Number of data points in the train set: 111127, number of used features: 172


In [75]:
train, test = pt.get_train_data(test_size=0)


pe finish
pe regularizrd



hr finish


In [81]:
pt.learn_lgb2(train,lgbm_params)

/Users/rince/Library/Python/3.7/lib/python/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] Total Bins 46312
[LightGBM] [Info] Number of data points in the train set: 88842, number of used features: 172


# 東京

In [120]:
base  = '2022050304'
pt.predict(base+'10')


              馬番    scores
202205030410   6  1.371102
202205030410   3  0.779867
202205030410   8  0.443859
202205030410   2  0.121382
202205030410   5 -0.451744
202205030410  14 -0.532018
202205030410  10 -0.581835
202205030410  11 -0.688470
202205030410   4 -0.755791
202205030410   9 -0.825304
202205030410  12 -0.914896
202205030410  13 -0.947285
202205030410  15 -1.005432
202205030410   1 -1.042639
202205030410   7 -1.054250


# 中京

In [119]:
base  = '2022070404'
# for i in ['07','08','09','10','11']:
pt.predict(base+'10')


              馬番    scores
202207040410  10  0.893336
202207040410   2  0.273061
202207040410   4 -0.812102
202207040410   8 -0.930210
202207040410   7 -0.981755
202207040410   9 -1.017185
202207040410   5 -1.021039
202207040410   1 -1.025512
202207040410   6 -1.042639
202207040410   3 -1.066951


# 函館

In [ ]:
base  = '2022020102'
# for i in ['07','08','09','10','11']:
pt.predict(base+'11')

In [ ]:

pt.show_results_today()

# 日付に注意

In [4]:
date = '2022/12/31'

# Results scraping

In [5]:
# race_id_list = results.index.astype('str')

results_tmp = Results.scrape(race_id_list)

  0%|          | 0/144 [00:00<?, ?it/s]

# Horse_results scraping

In [6]:
horse_id_list = results_tmp['horse_id'].astype(str).unique()
horse_results_tmp = HorseResults.scrape(horse_id_list)

# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'

  0%|          | 0/1902 [00:00<?, ?it/s]

# Peds scraping

In [7]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
peds_tmp = pe_2021.peds_re

  0%|          | 0/1902 [00:00<?, ?it/s]

  0%|          | 0/1902 [00:00<?, ?it/s]

# Return scraping

In [8]:
returns_tmp = Return.scrape(race_id_list)


  0%|          | 0/144 [00:00<?, ?it/s]

# update

In [9]:
new_results = update_data(load_csv(path_win+'results.csv'),results_tmp)
new_horse_results = update_data(load_csv(path_win+'horse_results.csv'),horse_results_tmp)
new_peds = update_data(load_csv(path_win+'peds.csv'),peds_tmp)
returns = load_csv(path_win+'return.csv')
returns.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
returns_tmp.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
new_return = update_data(returns,returns_tmp)


# save

In [10]:
new_results.to_csv(path_win+'results.csv')
new_horse_results.to_csv(path_win+'horse_results.csv')
new_peds.to_csv(path_win+'peds.csv')
new_return.to_csv(path_win+'return.csv')

# 重要度

In [ ]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習


教師あり, 教師なしでも生成されるベクトルは等しい

# model_ft 作成

In [ ]:
# 相対パスしかできない
# dim : 出力の次元
# minn : n_gramの最小単位
# maxn : n_gramの最大単位
path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
# 上書き保存OK
peds.to_csv(path_ft,header=False,index=False,sep=',')
model_ft = ft.train_unsupervised(path_ft,dim=62,minn=2,maxn=14)

model[model.words[1]] と model.get_input_vector(ind=1) は等価